In [ ]:
!pip install mecab-python3

In [ ]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 1s (194 kB/s)
Reading package lists... Done


In [ ]:
!apt-get install g++ openjdk-8-jdk python-dev python3-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
python3-dev is already the newest version (3.6.7-1~18.04).
openjdk-8-jdk is already the newest version (8u265-b01-0ubuntu2~18.04).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.


In [ ]:
!pip3 install JPype1-py3

In [ ]:
!pip3 install konlpy

In [ ]:
!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
# 이건 우선 패스할 것!
# 만약에 밑에까지 다 했는데, 오류 뜨면 이거 돌려보고 다시 해볼 것!
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 1s (200 kB/s)
Reading package lists... Done
Reading package lists... Done
Building

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Mecab
import numpy as np
import re
import pandas as pd
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
cafe_data = pd.read_excel("/content/drive/My Drive/Colab Notebooks/데이터/F 카페(7,859).xlsx")
# cafe_data.head()

,SPEAKER,SENTENCE,DATAID,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스
0,고객,아이스아메리카노 하나요,16334604,F,카페,카페,1,1,일반주문,NaN,Q,NaN,아이스 아메리카노 하나요,NaN,NaN,NaN,"아이스 아메리카노, 하나",NaN,NaN
1,점원,테이크아웃하실 건가요?,16334604,F,카페,카페,0,2,일반주문,테이크아웃여부,Q,NaN,NaN,테이크아웃하실 건가요?,NaN,NaN,테이크아웃,NaN,NaN
2,고객,네 텀블러에 넣어주세요,16334604,F,카페,카페,1,3,텀블러사용,NaN,Q,NaN,네 텀블러에 넣어주세요,NaN,NaN,NaN,텀블러,NaN,NaN
3,점원,텀블러 할인 300원 해드릴게요,16334604,F,카페,카페,0,4,텀블러사용,NaN,A,NaN,NaN,NaN,NaN,텀블러 할인 300원 해드릴게요,"텀블러, 할인, 300원",NaN,NaN
4,고객,그란데 사이즈로 주세요,16334604,F,카페,카페,1,5,일반주문,NaN,Q,NaN,그란데 사이즈로 주세요,NaN,NaN,NaN,그란데 사이즈,NaN,NaN


In [ ]:
# 단어 아니면 삭제
cafe_data['SENTENCE'] = cafe_data['SENTENCE'].str.replace("[^\w]|br", " ")
# cafe_data['text'] = cafe_data['text'].str.replace("[^\w]|br", " ")
# cafe_data.head()
# cafe_data.tail()

,SPEAKER,SENTENCE,DATAID,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스
0,고객,아이스아메리카노 하나요,16334604,F,카페,카페,1,1,일반주문,NaN,Q,NaN,아이스 아메리카노 하나요,NaN,NaN,NaN,"아이스 아메리카노, 하나",NaN,NaN
1,점원,테이크아웃하실 건가요,16334604,F,카페,카페,0,2,일반주문,테이크아웃여부,Q,NaN,NaN,테이크아웃하실 건가요?,NaN,NaN,테이크아웃,NaN,NaN
2,고객,네 텀블러에 넣어주세요,16334604,F,카페,카페,1,3,텀블러사용,NaN,Q,NaN,네 텀블러에 넣어주세요,NaN,NaN,NaN,텀블러,NaN,NaN
3,점원,텀블러 할인 300원 해드릴게요,16334604,F,카페,카페,0,4,텀블러사용,NaN,A,NaN,NaN,NaN,NaN,텀블러 할인 300원 해드릴게요,"텀블러, 할인, 300원",NaN,NaN
4,고객,그란데 사이즈로 주세요,16334604,F,카페,카페,1,5,일반주문,NaN,Q,NaN,그란데 사이즈로 주세요,NaN,NaN,NaN,그란데 사이즈,NaN,NaN


In [ ]:
pre_cafe_data = cafe_data.loc[0:6673,["SPEAKER","SENTENCE"]]

In [ ]:
# 점원과 고객이 한말 두개 따로 나누기.
# Q and A 형식으로 dataframe 만들기.

user = pre_cafe_data[pre_cafe_data["SPEAKER"] == "고객"]["SENTENCE"]
service = pre_cafe_data[pre_cafe_data["SPEAKER"] == "점원"]["SENTENCE"]
# display(user)
# display(service)
# print(type(user))

0           아이스아메리카노 하나요
2           네 텀블러에 넣어주세요
4           그란데 사이즈로 주세요
6         네 보고쿠폰으로 결제할게요
8                  괜찮습니다
              ...       
6664        비엔나는 어떤 거에요 
6666    카페라떼 아이스로 하나 주세요
6668     네 얼음은 조금만 넣어주세요
6670         원래하던데로 해주세요
6672        주차장은 앞에 있나요 
Name: SENTENCE, Length: 3337, dtype: object

1            테이크아웃하실 건가요 
3       텀블러 할인 300원 해드릴게요
5              적립카드 있으세요 
7            현금영수증 해드릴까요 
9         저쪽에서 잠시만 대기해주세요
              ...        
6665     아메리카노에 휘핑 들어갔습니다
6667         가지고 가시는 거에요 
6669         샷 더 넣어 드릴까요 
6671    연하다 싶으면 더 넣어 드릴게요
6673        주차장은 뒤에도 있습니다
Name: SENTENCE, Length: 3337, dtype: object

<class 'pandas.core.series.Series'>


In [ ]:
user.name="user"
service.name="service"

In [ ]:
service = pd.DataFrame(service).reset_index(drop=True)
user = pd.DataFrame(user).reset_index(drop=True)
result_data = pd.concat([user, service], axis=1)
# result_data

,user,service
0,아이스아메리카노 하나요,테이크아웃하실 건가요
1,네 텀블러에 넣어주세요,텀블러 할인 300원 해드릴게요
2,그란데 사이즈로 주세요,적립카드 있으세요
3,네 보고쿠폰으로 결제할게요,현금영수증 해드릴까요
4,괜찮습니다,저쪽에서 잠시만 대기해주세요
...,...,...
3332,비엔나는 어떤 거에요,아메리카노에 휘핑 들어갔습니다
3333,카페라떼 아이스로 하나 주세요,가지고 가시는 거에요
3334,네 얼음은 조금만 넣어주세요,샷 더 넣어 드릴까요
3335,원래하던데로 해주세요,연하다 싶으면 더 넣어 드릴게요


In [ ]:
tokenizer = Mecab()

# yeswords = ['NNG', 'NNP', 'NNB', 'NNBC', 'NR', 'NP', 'VV', 'VA', 'VX', 'VCP', 'VCN', 'MM', 'MAG', 'MAJ', 'IC']

encoder_input = []
for line in result_data["user"]:
    token = []
    words = tokenizer.pos(line)
    for word in words:
        # if word[1] in yeswords:
        token.append(word[0])
    encoder_input.append(token)

decoder_input = []
for line in result_data["service"]:
    token = []
    words = tokenizer.pos(line)
    for word in words:
        # if word[1] in yeswords:
        token.append(word[0])
    token.insert(0, "<start>")
    decoder_input.append(token)

decoder_output = []
for line in result_data["service"]:
    token = []
    words = tokenizer.pos(line)
    for word in words:
        # if word[1] in yeswords:
        token.append(word[0])
    token.append("<end>")
    decoder_output.append(token)
            

In [ ]:
# # 데이터 길이 자체가 짧기 때문에 그냥 split을 활용해도 충분할거 같은 느낌..?

# encoder_input = []
# for line in result_data["user"]:
#     token = []
#     words = line.split()
#     for word in words:
#         token.append(word)
#     encoder_input.append(token)

# # print(len(encoder_input))

# decoder_input = []
# for line in result_data["service"]:
#     token = []
#     words = line.split()
#     for word in words:
#         token.append(word)
#     token.insert(0, "<start>")
#     decoder_input.append(token)
# # print(len(decoder_input))

# decoder_output = []
# for line in result_data["service"]:
#     token=[]
#     words = line.split()
#     for word in words:
#         token.append(word)
#     token.append("<end>")
#     decoder_output.append(token)

# # print(len(decoder_output))

In [ ]:
print(encoder_input[:3])
print(decoder_input[:3])
print(decoder_output[:3])

[['아이스', '아메리카노', '하나', '요'], ['네', '텀블러', '에', '넣', '어', '주', '세요'], ['그란데', '사이즈', '로', '주', '세요']]
[['<start>', '테이크아웃', '하', '실', '건가요'], ['<start>', '텀블러', '할', '인', '300', '원', '해', '드릴게요'], ['<start>', '적립', '카드', '있', '으세요']]
[['테이크아웃', '하', '실', '건가요', '<end>'], ['텀블러', '할', '인', '300', '원', '해', '드릴게요', '<end>'], ['적립', '카드', '있', '으세요', '<end>']]


In [ ]:
tokenizer_text = Tokenizer()
tokenizer_text.fit_on_texts(encoder_input)
encoder_input = tokenizer_text.texts_to_sequences(encoder_input)

tokenizer_title = Tokenizer()
tokenizer_title.fit_on_texts(decoder_input)
tokenizer_title.fit_on_texts(decoder_output)
decoder_input = tokenizer_title.texts_to_sequences(decoder_input)
decoder_output = tokenizer_title.texts_to_sequences(decoder_output)

print(len(encoder_input))
print(encoder_input[:3])
print(decoder_input[:3])
print(decoder_output[:3])

3337
[[23, 10, 30, 8], [9, 102, 6, 45, 37, 1, 3], [285, 35, 17, 1, 3]]
[[1, 114, 8, 97, 147], [1, 210, 226, 299, 356, 10, 22, 32], [1, 49, 72, 7, 63]]
[[114, 8, 97, 147, 2], [210, 226, 299, 356, 10, 22, 32, 2], [49, 72, 7, 63, 2]]


In [ ]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_output = pad_sequences(decoder_output, padding="post")

In [ ]:
title_to_index = tokenizer_title.word_index
index_to_title = tokenizer_title.index_word

In [ ]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_output.shape)

print(len(encoder_input))
print(len(decoder_input))
print(len(decoder_output))

(3337, 24)
(3337, 34)
(3337, 34)
3337
3337
3337


In [ ]:
import pickle
with open('title_to_index.pickle','wb') as fw:
    pickle.dump(title_to_index, fw)
with open("index_to_title.pickle", "wb") as fw:
    pickle.dump(index_to_title, fw)

ValueError: ignored

In [ ]:
test_size = 800
encoder_input_train = encoder_input[ :-test_size]
decoder_input_train = decoder_input[ :-test_size]
decoder_output_train = decoder_output[ :-test_size]

encoder_input_test = encoder_input[test_size: ]
decoder_input_test = decoder_input[test_size: ]
decoder_output_test = decoder_output[test_size: ]

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
encoder_inputs = Input(shape=(24,))
encoder_embed = Embedding(len(tokenizer_text.word_index)+1, 120)(encoder_inputs)
encoder_mask = Masking(mask_value=0)(encoder_embed)
encoder_outputs, h_state, c_state = LSTM(120, return_sequences=True, return_state=True)(encoder_mask)

In [ ]:
decoder_inputs = Input(shape=(34,))
decoder_embed = Embedding(len(tokenizer_title.word_index)+1, 120)(decoder_inputs)
decoder_mask = Masking(mask_value=0)(decoder_embed)
decoder_lstm = LSTM(120, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_mask, initial_state=[h_state, c_state])

In [ ]:
from attention import AttentionLayer
attn_layer = AttentionLayer()
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

decoder_concat_input = Concatenate()([decoder_outputs, attn_out])

decoder_dense = Dense(len(tokenizer_title.word_index)+1, activation= 'softmax')
decoder_softmax_outputs = decoder_dense(decoder_concat_input)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.fit(x=[encoder_input_train, decoder_input_train],
          y=decoder_output_train,
          callbacks=[es],
          validation_data=([encoder_input_test, decoder_input_test], decoder_output_test),
          batch_size=128, epochs=150)

Epoch 1/150
20/20 [==============================] - ETA: 0s - loss: 2.9009 - acc: 0.7216
Epoch 00001: val_acc improved from -inf to 0.74459, saving model to best_model.h5
20/20 [==============================] - 4s 217ms/step - loss: 2.9009 - acc: 0.7216 - val_loss: 1.6172 - val_acc: 0.7446
Epoch 2/150
20/20 [==============================] - ETA: 0s - loss: 1.4559 - acc: 0.7667
Epoch 00002: val_acc improved from 0.74459 to 0.75414, saving model to best_model.h5
20/20 [==============================] - 2s 103ms/step - loss: 1.4559 - acc: 0.7667 - val_loss: 1.4951 - val_acc: 0.7541
Epoch 3/150
20/20 [==============================] - ETA: 0s - loss: 1.3370 - acc: 0.7726
Epoch 00003: val_acc improved from 0.75414 to 0.75688, saving model to best_model.h5
20/20 [==============================] - 2s 97ms/step - loss: 1.3370 - acc: 0.7726 - val_loss: 1.3902 - val_acc: 0.7569
Epoch 4/150
20/20 [==============================] - ETA: 0s - loss: 1.2541 - acc: 0.7815
Epoch 00004: val_acc impro

In [ ]:
encoder_model = Model(encoder_inputs, [encoder_outputs, h_state, c_state])

encoder_h_state = Input(shape=(120,))
encoder_c_state = Input(shape=(120,))

pd_decoder_outputs,pd_h_state,pd_c_state = decoder_lstm(decoder_mask,
                                                        initial_state=[encoder_h_state,encoder_c_state])

pd_encoder_outputs = Input(shape=(24,120))
pd_attn_out, pd_attn_states = attn_layer([pd_encoder_outputs, pd_decoder_outputs])
pd_decoder_concat = Concatenate()([pd_decoder_outputs, pd_attn_out])

pd_decoder_softmax_outputs = decoder_dense(pd_decoder_concat)

decoder_model = Model([decoder_inputs, pd_encoder_outputs, encoder_h_state, encoder_c_state],
                      [pd_decoder_softmax_outputs, pd_h_state, pd_c_state])

In [ ]:
with open('encoder_model.json', 'w', encoding='utf8') as f:
    f.write(encoder_model.to_json())
encoder_model.save_weights('encoder_model_weights.h5')

with open('decoder_model.json', 'w', encoding='utf8') as f:
    f.write(decoder_model.to_json())
decoder_model.save_weights('decoder_model_weights.h5')

In [ ]:
input_stc = input()

token_stc = []
for line in tokenizer.pos(input_stc):
    # if line[1] in yeswords:
    token_stc.append(line[0])

# token_stc = input_stc.split()
encode_stc = tokenizer_text.texts_to_sequences([token_stc])
pad_stc = pad_sequences(encode_stc, maxlen=34, padding='post')

en_out, en_hidden, en_cell = encoder_model.predict(pad_stc)

predicted_seq = np.zeros((1,1))
predicted_seq[0,0] = title_to_index["<start>"]

decoded_stc = []

while True:
    output_words, h, c = decoder_model.predict([predicted_seq, en_out, en_hidden, en_cell])
    predicted_word = index_to_title[np.argmax(output_words[0,0])]
    
    if predicted_word == "<end>":
        break
    
    decoded_stc.append(predicted_word)

    predicted_seq = np.zeros((1,1))
    predicted_seq[0, 0] = np.argmax(output_words[0,0])

    en_hidden = h
    en_cell = c

print(" ".join(decoded_stc))

차가운 아메리카노요
네 포인트 적립 하 시 면 커피 와 드리 겠 습니다
